**Load Data**

In [174]:
# Useful imports
import os
import re
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.model_selection import train_test_split

# Useful globals
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [142]:
# Where to find the data
base_dir = "./data"

def load_data(language):
    """
        Loads the books to be processed
        and returns them as a list of strings.

        Input:
            string: Language whose books to load
            number: Number of books to process

        Ouput:
            list: List of strings of the loaded books
    """
    # Output
    books = []

    # Working path
    wrk_path = "{}/{}".format(base_dir, language)

    # Books' filenames
    book_files = os.listdir(wrk_path)

    # Read each book and append
    for book_file in book_files:
        input_file = "{}/{}".format(wrk_path, book_file)
        with open(input_file) as f:
            book = f.read()
            books.append(book)

    # Useful logs about books
    print("Loaded {} books for {}.".format(len(books), language))
    for i in range(len(books)):
        print("There are {} words in {}.".format(len(books[i].split()), book_files[i]))

    return books

In [148]:
books = load_data("english")

Loaded 5 books for english.
There are 74964 words in book2.txt.
There are 26436 words in book3.txt.
There are 121561 words in book1.txt.
There are 211862 words in book4.txt.
There are 25572 words in book6.txt.


**Data Preparation**

In [144]:
def clean_book(book):
    """
        Remove unwanted special characters
        and extra spaces from the text.
        
        Input:
            list: List of books
        Output:
            list: Cleaned list of books
    """
    book = re.sub(r'\n', ' ', book) 
    book = re.sub(r'[{}@_*>()\\#%+=\[\]]','', book)
    book = re.sub('a0','', book)
    book = re.sub('\'92t','\'t', book)
    book = re.sub('\'92s','\'s', book)
    book = re.sub('\'92m','\'m', book)
    book = re.sub('\'92ll','\'ll', book)
    book = re.sub('\'91','', book)
    book = re.sub('\'92','', book)
    book = re.sub('\'93','', book)
    book = re.sub('\'94','', book)
    book = re.sub('\.','. ', book)
    book = re.sub('\!','! ', book)
    book = re.sub('\?','? ', book)
    book = re.sub(' +',' ', book)
    return book

In [149]:
# Clean books
clean_books = []
for book in books:
    clean_books.append(clean_book(book))

In [150]:
# Dictionary for vacabolary
vocab_to_int = {}
count = 0
for book in clean_books:
    for character in book:
        if character not in vocab_to_int:
            vocab_to_int[character] = count
            count += 1

In [152]:
# Check the size of vocabulary and all of the values
vocab_size = len(vocab_to_int)
print("The vocabulary contains {} characters.".format(vocab_size))
print(sorted(vocab_to_int))

The vocabulary contains 87 characters.
[' ', '!', '$', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', 'â', 'æ', 'è', 'é', 'ê', 'ô', 'œ', '—', '‘', '’', '“', '”']


In [153]:
# Int to vocab
int_to_vocab = {}
for character, value in vocab_to_int.items():
    int_to_vocab[value] = character

In [154]:
# Create sentences from the books
sentences = []
for book in clean_books:
    for sentence in book.split(". "):
        sentences.append(sentence + ".")
print("There are {} sentences".format(len(sentences)))

There are 18948 sentences


In [97]:
# # Convert sentences to integers
# int_sentences = []
# for sentence in sentences:
#     int_sentence = []
#     for character in sentence:
#         int_sentence.append(vocab_to_int[character])
#     int_sentences.append(int_sentence)

In [100]:
# # Sentence's length
# lengths = []
# for sentence in int_sentences:
#     lengths.append(len(sentence))
# lengths = pd.DataFrame(lengths, columns=["counts"])
# lengths.describe()

In [155]:
# Limit data for training
max_length = 92
min_length = 10

good_sentences = []

for sentence in sentences:
    if len(sentence) <= max_length and len(sentence) >= min_length:
        good_sentences.append(sentence)
print("We will use {} sentences to train and test our model.".format(len(good_sentences)))

We will use 7443 sentences to train and test our model.


In [156]:
# Noise maker (input sequences)
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
           'n','o','p','q','r','s','t','u','v','w','x','y','z',]

def noise_maker(sentence, threshold):
    '''Relocate, remove, or add characters to create spelling mistakes'''
    
    noisy_sentence = []
    i = 0
    while i < len(sentence):
        random = np.random.uniform(0,1,1)
        # Most characters will be correct since the threshold value is high
        if random < threshold:
            noisy_sentence.append(sentence[i])
        else:
            new_random = np.random.uniform(0,1,1)
            # ~33% chance characters will swap locations
            if new_random > 0.67:
                if i == (len(sentence) - 1):
                    # If last character in sentence, it will not be typed
                    continue
                else:
                    # if any other character, swap order with following character
                    noisy_sentence.append(sentence[i+1])
                    noisy_sentence.append(sentence[i])
                    i += 1
            # ~33% chance an extra lower case letter will be added to the sentence
            elif new_random < 0.33:
                random_int = np.random.randint(0, len(int_to_vocab)-1)
                noisy_sentence.append(int_to_vocab[random_int])
                noisy_sentence.append(sentence[i])
            # ~33% chance a character will not be typed
            else:
                pass     
        i += 1
    return noisy_sentence

In [157]:
# Check to ensure noise_maker is making mistakes correctly.
threshold = 0.9
for sentence in good_sentences[:5]:
    print(sentence)
    print(''.join(noise_maker(sentence, threshold)))
    print()

Inspirited by this wind of promise, my daydreams become more fervent and vivid.
InspirHitde by t—his wind of pr-omise, ydaydreams beome more ferevnta nd vivid.

This expedition has been the favourite dream of my early years.
hTsi epxedition hsa been thef avourie dream of my early years.

My education was neglected, yet I was passionately fond of reading.
My ducaionL was neglected, yet I was passionateyl fon of reading.

You are well acquainted with my failure and how heavily I bore the disappointment.
You are well acquainted with my faliure nad hw hevavily TI bore the dsiapponitment.

Six years have passed since I resolved on my present undertaking.
Six y?ears hae passd since I resovled no my rpesent u:ndertaking.



In [172]:
# Create input and target text
input_texts = []
target_texts = []

input_characters = set()
target_characters = set()

# Shuffle the list of good_senteces
np.random.shuffle(good_sentences)

# Pad target sentences and build
# input and target vocabulary
for sentence in good_sentences:
    input_text = '\t' + "".join(noise_maker(sentence, 0.9)) + '\n'
    input_texts.append(input_text)
    target_texts.append(sentence)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in sentence:
        if char not in target_characters:
            target_characters.add(char)

# Set up our encoder and decoder inputs
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Logging
# print('Number of samples:', len(input_texts))
# print('Number of unique input tokens:', num_encoder_tokens)
# print('Number of unique output tokens:', num_decoder_tokens)
# print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Max sequence length for outputs:', max_decoder_seq_length)

# Create dicts for the input and
# target vocabolaries
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# Crete vectorize data placeholders 
# for encoder and decoder
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

**Build Model**

In [173]:
# Populate tensors
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [175]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [176]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [177]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [178]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 5954 samples, validate on 1489 samples
Epoch 1/100
5954/5954 [==============================] - 72s 12ms/step - loss: 1.7490 - val_loss: 1.6949
Epoch 2/100
5954/5954 [==============================] - 70s 12ms/step - loss: 1.5537 - val_loss: 1.4940
Epoch 3/100
5954/5954 [==============================] - 69s 12ms/step - loss: 1.3992 - val_loss: 1.3621
Epoch 4/100
5954/5954 [==============================] - 69s 12ms/step - loss: 1.3141 - val_loss: 1.3184
Epoch 5/100
5954/5954 [==============================] - 69s 12ms/step - loss: 1.2541 - val_loss: 1.2455
Epoch 6/100
5954/5954 [==============================] - 460s 77ms/step - loss: 1.2053 - val_loss: 1.2158
Epoch 7/100
5954/5954 [==============================] - 70s 12ms/step - loss: 1.1646 - val_loss: 1.1815
Epoch 8/100
5954/5954 [==============================] - 70s 12ms/step - loss: 1.1295 - val_loss: 1.1830
Epoch 9/100
5954/5954 [==============================] - 70s 12ms/step - loss: 1.0994 - val_loss: 1.1216
Epoch 

Epoch 78/100
5954/5954 [==============================] - 80s 13ms/step - loss: 0.4840 - val_loss: 1.0135
Epoch 79/100
5954/5954 [==============================] - 80s 13ms/step - loss: 0.4797 - val_loss: 1.0212
Epoch 80/100
5954/5954 [==============================] - 71s 12ms/step - loss: 0.4762 - val_loss: 1.0287
Epoch 81/100
5954/5954 [==============================] - 71s 12ms/step - loss: 0.4721 - val_loss: 1.0248
Epoch 82/100
5954/5954 [==============================] - 71s 12ms/step - loss: 0.4682 - val_loss: 1.0324
Epoch 83/100
5954/5954 [==============================] - 79s 13ms/step - loss: 0.4640 - val_loss: 1.0563
Epoch 84/100
5954/5954 [==============================] - 81s 14ms/step - loss: 0.4616 - val_loss: 1.0378
Epoch 85/100
5954/5954 [==============================] - 79s 13ms/step - loss: 0.4575 - val_loss: 1.0602
Epoch 86/100
5954/5954 [==============================] - 80s 14ms/step - loss: 0.4543 - val_loss: 1.0695
Epoch 87/100
5954/5954 [======================

In [179]:
# Save model
model.save('charSeq2Seq.h5')

/Users/dailand10/Desktop/dlbaby/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).


**Inference**

In [180]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [184]:
target_token_index["\"]

SyntaxError: EOL while scanning string literal (<ipython-input-184-3ca34360fce3>, line 1)

In [181]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

KeyError: '\t'

In [185]:
target_token_index.items()

dict_items([(' ', 0), ('!', 1), ('&', 2), ("'", 3), (',', 4), ('-', 5), ('.', 6), ('0', 7), ('1', 8), ('2', 9), ('3', 10), ('4', 11), ('5', 12), ('6', 13), ('7', 14), ('8', 15), ('9', 16), (':', 17), (';', 18), ('?', 19), ('A', 20), ('B', 21), ('C', 22), ('D', 23), ('E', 24), ('F', 25), ('G', 26), ('H', 27), ('I', 28), ('J', 29), ('K', 30), ('L', 31), ('M', 32), ('N', 33), ('O', 34), ('P', 35), ('Q', 36), ('R', 37), ('S', 38), ('T', 39), ('U', 40), ('V', 41), ('W', 42), ('X', 43), ('Y', 44), ('Z', 45), ('`', 46), ('a', 47), ('b', 48), ('c', 49), ('d', 50), ('e', 51), ('f', 52), ('g', 53), ('h', 54), ('i', 55), ('j', 56), ('k', 57), ('l', 58), ('m', 59), ('n', 60), ('o', 61), ('p', 62), ('q', 63), ('r', 64), ('s', 65), ('t', 66), ('u', 67), ('v', 68), ('w', 69), ('x', 70), ('y', 71), ('z', 72), ('æ', 73), ('—', 74), ('‘', 75), ('’', 76), ('“', 77), ('”', 78)])